In [11]:
from langchain.tools import tool

In [17]:
import os
from langchain import hub
from langchain.agents import AgentExecutor
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.tools.render import render_text_description

#for Bing
import os
from langchain.utilities import BingSearchAPIWrapper, SerpAPIWrapper

# for Agent
from langchain.agents import AgentExecutor, BaseSingleActionAgent, Tool, ZeroShotAgent
from langchain.schema import AgentAction, AgentFinish

# for LLM
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Maintenance
from typing import Any, List, Tuple, Union

# for Chain
from langchain.chains import LLMChain




class CognitiveFeedbackAgent:
    def __init__(self):
        # Initialize the LLM
        self.llm = LlamaCpp(
            model_path="llama-2-7b-chat.Q6_K.gguf",
            temperature=0.1,
            max_tokens=500000,
            top_p=0.9,
            callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
            verbose=True,
        )

        # Set up the environment variables for Bing
        os.environ["BING_SUBSCRIPTION_KEY"] = "02fa2d164e534dd69f2a586caba57cb6"
        os.environ["BING_SEARCH_URL"] = "https://api.bing.microsoft.com/v7.0/search"

        # Initialize Bing API and other components
        bing_API_wrapper = BingSearchAPIWrapper()
        bing_tool = Tool(
            name="Bing",
            func=bing_API_wrapper.run,
            description="useful for when you need descriptive answers to questions about current state of affairs",
            return_direct=False)

        self.SERP_API_KEY = "d38482db0efe9c48ec77ffdd165c3fb4d7144a6bf86b3604e4367b0a0fe17fdd"
        self.google_params = {
            "engine": "google",
            "gl": "uk",
            "hl": "en"}
        self.google_API_wrapper = SerpAPIWrapper(serpapi_api_key=SERP_API_KEY, params=google_params)
        self.google_tool =  Tool(
            name="Internet",
            func=google_API_wrapper.run,
            description="Useful for when you need short answers (e.g., a single number) to questions about current state of affairs",
            return_direct=True)
        
        self.tools = [self.google_tool]

        # Define the prompt for the LLMChain
        self.prompt = ZeroShotAgent.create_prompt(
            self.tools,
            prefix="Determine if the following question can be answered with your current knowledge. If not, you have access to following online search tools. You may try to break down the question to steps.",
            suffix="'Begin!\n\nQuestion: {input}\nThought:{agent_scratchpad}',",
            input_variables=["input", "agent_scratchpad"]
        )

        # Create the LLMChain with OpenAI model
        self.llm_chain = LLMChain(llm=self.llm, prompt=self.prompt)
        
        # Define the agent and agent executor
        self.tool_names = [tool.name for tool in self.tools]
        self.agent = ZeroShotAgent(llm_chain=self.llm_chain, allowed_tools=self.tool_names)
        self.agent_executor = AgentExecutor.from_agent_and_tools(agent=self.agent, tools=self.tools, verbose=True)
    
    def classify_communication(self, input_text):
        if "?" in input_text:
            return 'factual question'
        else:
            return 'other'

    def handle_communication(self, input_text, communication_type):
        if communication_type == 'factual question':
            response = self.agent_executor.invoke({"input": input_text})
        else:
            response = "I'm not sure how to handle this type of communication."
        return response

    def run(self):
        while True:
            input_text = input("User: ")
            communication_type = self.classify_communication(input_text)
            response = self.handle_communication(input_text, communication_type)
            print(f"CFA: {response}")

if __name__ == "__main__":
    cfa = CognitiveFeedbackAgent()
    cfa.run()

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


KeyboardInterrupt: Interrupted by user

In [15]:
@tool
def search_tool() -> str:
    """ """
    SERP_API_KEY = "d38482db0efe9c48ec77ffdd165c3fb4d7144a6bf86b3604e4367b0a0fe17fdd"
    google_params = {
            "engine": "google",
            "gl": "uk",
            "hl": "en"}
    google_API_wrapper = SerpAPIWrapper(serpapi_api_key=SERP_API_KEY, params=google_params)
    google_tool =  Tool(
        name="Internet",
        func=google_API_wrapper.run,
        description="Useful for when you need short answers (e.g., a single number) to questions about current state of affairs",
        return_direct=True)

    return google_tool 
            
tools = [search_tool()]


TypeError: __call__() missing 1 required positional argument: 'tool_input'

In [8]:
google_API_wrapper.run("Current US president?")

'Joe Biden'

In [14]:
tools

[Tool(name='Internet', description='Useful for when you need short answers (e.g., a single number) to questions about current state of affairs', return_direct=True, func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'gl': 'uk', 'hl': 'en'}, serpapi_api_key='d38482db0efe9c48ec77ffdd165c3fb4d7144a6bf86b3604e4367b0a0fe17fdd', aiosession=None)>)]

In [3]:
cfa.__dict__

{'llm': LlamaCpp(callbacks=<langchain.callbacks.manager.CallbackManager object at 0x0000021B14CC2160>, client=<llama_cpp.llama.Llama object at 0x0000021B14CC21C0>, model_path='llama-2-7b-chat.Q6_K.gguf', max_tokens=500000, temperature=0.1, top_p=0.9),
 'factual_question_agent': {
   input: RunnableLambda(...),
   agent_scratchpad: RunnableLambda(...)
 }
 | PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': 'Bing: useful for when you need descriptive answers to questions about current state of affairs', 'tool_names': 'Bing'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can